# Task 1

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt

In [ ]:
#Importing Feature Geometry Shapefiles
catchments_primary = gpd.read_file("Catchments/catchments_primary.shp")
catchments_secondary = gpd.read_file("Catchments/catchments_secondary.shp")
catchments_future = gpd.read_file("Catchments/catchments_future.shp")
sa = gpd.read_file("SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp")
unemployment = gpd.read_file("Data/SA2_unemployment.shp")

In [ ]:
#Importing csv, txt files
businesses=pd.read_csv("Businesses.csv")
income=pd.read_csv("Income.csv")
polling=pd.read_csv("PollingPlaces2019.csv")
population=pd.read_csv("Population.csv")

stops=pd.read_csv("Stops.txt", sep=",")

In [ ]:
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras
import json

credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        try:
            db = create_engine('postgresql+psycopg2://'+db_user+':'+db_pw+'@'+host+'/'+default_db, echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(sqlcmd, args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

In [ ]:
db, conn = pgconnect(credentials)

Connected successfully.


In [ ]:
unemployment

,SA2_MAIN,SA2_MAIN16,SA2_NAME,STATE_CODE,STATE_NAME,AREA_SQKM,PER_UNEMPL,geometry
0,101021007,101021007,Braidwood,1,New South Wales,3418.3525,4.2,"POLYGON ((149.56837 -36.00876, 149.56813 -36.0..."
1,101021008,101021008,Karabar,1,New South Wales,6.9825,4.7,"POLYGON ((149.23877 -35.36226, 149.23867 -35.3..."
2,101021009,101021009,Queanbeyan,1,New South Wales,4.7634,5.2,"POLYGON ((149.23877 -35.36226, 149.23764 -35.3..."
3,101021010,101021010,Queanbeyan - East,1,New South Wales,13.0034,5.0,"POLYGON ((149.24148 -35.33667, 149.24308 -35.3..."
4,101021011,101021011,Queanbeyan Region,1,New South Wales,3054.4099,2.9,"POLYGON ((149.76065 -35.08348, 149.75962 -35.0..."
...,...,...,...,...,...,...,...,...
571,128021537,128021537,Royal National Park,1,New South Wales,139.3336,0.0,"POLYGON ((151.07257 -34.04744, 151.07293 -34.0..."
572,128021538,128021538,Sutherland - Kirrawee,1,New South Wales,7.7547,4.0,"POLYGON ((151.04088 -34.02826, 151.04163 -34.0..."
573,128021607,128021607,Engadine,1,New South Wales,8.9538,3.0,"POLYGON ((151.04044 -34.03608, 151.04041 -34.0..."
574,128021608,128021608,Loftus - Yarrawarrah,1,New South Wales,3.8436,3.0,"POLYGON ((151.04255 -34.03530, 151.04289 -34.0..."
